<a href="https://colab.research.google.com/github/MaximeGloesener/HandsOnAI-Challenge1/blob/master/4_Traitement_temps_r%C3%A9el.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ **I**. Récupération d'un modèle entrainé depuis un drive

✅ **II**. Récupération des vidéos de test depuis le serveur

✅ **III**. Prédictions sur la vidéo


# **1. Hardware Informations (GPU)**

In [1]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

/bin/bash: /opt/bin/nvidia-smi: No such file or directory


# **2. Importation des librairies**

In [9]:
import cv2
from keras.models import load_model
import numpy as np
import tensorflow as tf

#**3. Connexion à Google Colab pour accéder au modèle**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#**4. Téléchargement du modèle**


In [7]:
model = load_model('gdrive/MyDrive/Challenge1/test.h5')

#**5. Récupération des vidéos depuis le serveur**

In [6]:
!wget https://cluster.ig.umons.ac.be/HackIA21/databases/fire_detection/TestVideo1.avi

--2022-11-24 23:11:10--  https://cluster.ig.umons.ac.be/HackIA21/databases/fire_detection/TestVideo1.avi
Resolving cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)... 193.190.209.220
Connecting to cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)|193.190.209.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2305374 (2.2M) [video/x-msvideo]
Saving to: ‘TestVideo1.avi’

TestVideo1.avi      100%[===================>]   2.20M  3.55MB/s    in 0.6s    

2022-11-24 23:11:12 (3.55 MB/s) - ‘TestVideo1.avi’ saved [2305374/2305374]



# **6. Lancement des prédictions sur la vidéo**

In [10]:
video_cap = cv2.VideoCapture('TestVideo1.avi')
input_dim = 224
frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video_cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'DivX')
output = cv2.VideoWriter('output.mp4', fourcc, fps, (frame_width, frame_height))
classes = ["fire","no_fire","start_fire"]

while True:
        success, frame = video_cap.read()

        if success:

          img = cv2.resize(frame, (input_dim, input_dim), interpolation = cv2.INTER_AREA)

          # Resizing into dimensions you used while training
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          img = tf.keras.preprocessing.image.img_to_array(img)
          img = np.expand_dims(img, axis = 0)

          pred = model.predict(img, batch_size = 1)[0]

          # Annotate the frame
          for (pos,prob) in enumerate(pred):
              class_name = classes[pos]
              if (pos == np.argmax(pred)) :
                  font = cv2.FONT_HERSHEY_COMPLEX 
                  textsize = cv2.getTextSize(class_name, font, 1, 2)[0]
                  textX = (img.shape[1] - textsize[0]) / 3
                  textY = (img.shape[0] + textsize[1]) / 2
                  cv2.putText(frame, class_name, (int(textX)-10, int(textY)), font, 1, (255,255,255), 2, cv2.LINE_AA)
                  output.write(frame)


        else:
          break

video_cap.release()
output.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 274ms/step
